# Model GLM2 - ROI Analysis
### Comprehensive -- compares regional activation on fixed and conditional (correct vs. incorrect)

In [ ]:
from os.path import join
from glob import glob
from scipy import stats
from scipy.stats import pearsonr
from pylab import *
from math import sqrt
import nibabel as nb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("white")
sns.set_palette('muted')

subs = ['WMAZE_001', 'WMAZE_002', 'WMAZE_004', 'WMAZE_005', 'WMAZE_006', 
        'WMAZE_007', 'WMAZE_008', 'WMAZE_009', 'WMAZE_010', 'WMAZE_012', 
        'WMAZE_017', 'WMAZE_018', 'WMAZE_019', 'WMAZE_020', 'WMAZE_021',  
        'WMAZE_022', 'WMAZE_023', 'WMAZE_024', 'WMAZE_026', 'WMAZE_027']
proj_dir = '/home/data/madlab/data/mri/wmaze' 
mask_filenames = []
cope_files = []
for sub in subs:
    mask_filenames_glob = glob(proj_dir+ '/roi_analysis/anat_masks/_subject_id_'+ sub +'/_anatmask_xfm*/*')
    mask_filenames.append(sorted(mask_filenames_glob))
    subjcopes_glob = glob(proj_dir+ '/scndlvl/model_GLM2/'+ sub +'/fixedfx/cope_*')
    cope_files.append(sorted(subjcopes_glob))
    if len(cope_files[-1]) == 0:
        print(sub)

### Cell to double-check the array indexing

In [ ]:
#change first index of cope_files to indicate participant index in sids array
for i, curr_mask in enumerate(mask_filenames[0]):
    print(i, mask_filenames[0][i].split('/')[-1][:-7]) 

In [ ]:
for i, curr_cope in enumerate(cope_files[0]):
    print(i, cope_files[0][i].split('/')[-1][5:-7]) 

### Use binarized mask to obtain activation in left & right hemisphere for each region of interest

In [ ]:
all_data = {'subjid':[],
            'lhhp_fixed_corr':[], 'rhhp_fixed_corr':[], 
            'lhhp_fixed_incorr':[], 'rhhp_fixed_incorr':[],
            'lhhp_cond_corr':[], 'rhhp_cond_corr':[], 
            'lhhp_cond_incorr':[], 'rhhp_cond_incorr':[],
            'lhcaud_fixed_corr':[], 'rhcaud_fixed_corr':[], 
            'lhcaud_fixed_incorr':[], 'rhcaud_fixed_incorr':[],
            'lhcaud_cond_corr':[], 'rhcaud_cond_corr':[], 
            'lhcaud_cond_incorr':[], 'rhcaud_cond_incorr':[],
            'lhput_fixed_corr':[], 'rhput_fixed_corr':[], 
            'lhput_fixed_incorr':[], 'rhput_fixed_incorr':[],
            'lhput_cond_corr':[], 'rhput_cond_corr':[], 
            'lhput_cond_incorr':[], 'rhput_cond_incorr':[],
            'lhmpfc_fixed_corr':[], 'rhmpfc_fixed_corr':[], 
            'lhmpfc_fixed_incorr':[], 'rhmpfc_fixed_incorr':[],
            'lhmpfc_cond_corr':[], 'rhmpfc_cond_corr':[], 
            'lhmpfc_cond_incorr':[], 'rhmpfc_cond_incorr':[]}

for i in range(len(subs)):
    all_data['subjid'].append(subs[i])
    #ROI masks
    lh_hp_img = nb.load(mask_filenames[i][2])
    rh_hp_img = nb.load(mask_filenames[i][11])
    lh_mpfc_img = nb.load(mask_filenames[i][4])
    rh_mpfc_img = nb.load(mask_filenames[i][13])
    lh_caud_img = nb.load(mask_filenames[i][7])
    rh_caud_img = nb.load(mask_filenames[i][8])
    lh_put_img = nb.load(mask_filenames[i][16])
    rh_put_img = nb.load(mask_filenames[i][17])   
    #copes
    fixed_corr_img = nb.load(cope_files[i][20])
    fixed_incorr_img = nb.load(cope_files[i][21])
    cond_corr_img = nb.load(cope_files[i][14])
    cond_incorr_img = nb.load(cope_files[i][15]) 
    
    region = ['hp', 'mpfc', 'caud', 'put']
    learn_type = ['fixed_corr', 'fixed_incorr', 'cond_corr', 'cond_incorr']   
    for r in region:
        for l in learn_type:
            lh_data = eval('{0}_img.get_data()[lh_{1}_img.get_data() > 0.]'.format(l,r))
            all_data['lh{0}_{1}'.format(r,l)].append(np.mean(lh_data)) 
            rh_data = eval('{0}_img.get_data()[rh_{1}_img.get_data() > 0.]'.format(l,r))
            all_data['rh{0}_{1}'.format(r,l)].append(np.mean(rh_data))
    
all_data_df = pd.DataFrame(all_data)

### Combine two hemispheres to create one regional average

In [ ]:
region = ['hp', 'mpfc', 'caud', 'put']
learn_type = ['fixed_corr', 'fixed_incorr', 'cond_corr', 'cond_incorr']
for r in region:
    for l in learn_type:
        all_data_df['{0}_{1}'.format(r,l)] = (all_data_df['lh{0}_{1}'.format(r,l)] 
                                              + all_data_df['rh{0}_{1}'.format(r,l)])/2.

### Region-specific dataframes for producing plots

In [ ]:
hp_plot = {}
mpfc_plot = {}
caud_plot = {}
put_plot = {}

for i in ['hp', 'mpfc', 'caud', 'put']:
    exec('{0}_plot["fixed_corr"] = all_data_df["{0}_fixed_corr"]'.format(i))
    exec('{0}_plot["fixed_incorr"] = all_data_df["{0}_fixed_incorr"]'.format(i))
    exec('{0}_plot["cond_corr"] = all_data_df["{0}_cond_corr"]'.format(i))
    exec('{0}_plot["cond_incorr"] = all_data_df["{0}_cond_incorr"]'.format(i))
    exec('{0}_plot_df = pd.DataFrame.from_dict({0}_plot)'.format(i))

# Hippocampus

In [ ]:
hp_plot_df

In [ ]:
for key in hp_plot_df:
    print "Mean {0}: ".format(key), np.mean(hp_plot_df['{0}'.format(key)])
    print "STD {0}: ".format(key), np.std(hp_plot_df['{0}'.format(key)])
    print ""
N = 4
conditions = ['Fixed Corr', 'Fixed Incorr', 'Conditional Corr', 'Conditional Incorr']
hp_allsubjs = [hp_plot_df['fixed_corr'], hp_plot_df['fixed_incorr'],
               hp_plot_df['cond_corr'], hp_plot_df['cond_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (10,6))
ax = sns.boxplot(data = hp_allsubjs, color = "#278fea", width = 0.3)
ax = sns.swarmplot(data = hp_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Hippocampal Activation")
plt.show()

### Hippocampus - Fixed vs. Conditional

In [ ]:
all_fixed = hp_plot_df[['fixed_corr', 'fixed_incorr']].mean(axis=1)
all_fixed = np.array(all_fixed)
all_cond = hp_plot_df[['cond_corr', 'cond_incorr']].mean(axis=1)
all_cond = np.array(all_cond)
print "HPC FIXED", np.mean(all_fixed), np.std(all_fixed)
print "HPC COND", np.mean(all_cond), np.std(all_cond)
print ""
print 'Hippocampus Fixed vs. Conditional'
print stats.ttest_rel(all_fixed, all_cond)
cohens_d = ((np.average(all_fixed) - np.average(all_cond)) / (sqrt((np.std(all_fixed, ddof = 1)) 
            ** 2 + np.std(all_cond, ddof = 1) ** 2) / 2))
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Fixed', 'Conditional']
hp_allsubjs = [all_fixed, all_cond]
ind = np.arange(N)
width = 0.5
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = hp_allsubjs, color = "#278fea", width = 0.3)
ax = sns.swarmplot(data = hp_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("HPC Activation")
plt.show()

### Hippocampus Fixed - Correct vs. Incorrect

In [ ]:
cohens_d=((np.average(hp_plot_df['fixed_corr'])-np.average(hp_plot_df['fixed_incorr']))
          /(sqrt((np.std(hp_plot_df['fixed_corr'],ddof=1)) 
           **2+np.std(hp_plot_df['fixed_incorr'],ddof = 1)**2)/2))
print 'Hippocampus Fixed Correct vs. Fixed Incorrect'
print stats.ttest_rel(hp_plot_df['fixed_corr'], hp_plot_df['fixed_incorr'])
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Fixed Corr', 'Fixed Incorr']
hp_allsubjs = [hp_plot_df['fixed_corr'], hp_plot_df['fixed_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = hp_allsubjs, color = "#278fea", width = 0.3)
ax = sns.swarmplot(data = hp_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Hippocampal Activation")
plt.show()

### Hippocampus Conditional - Correct vs. Incorrect

In [ ]:
cohens_d=((np.average(hp_plot_df['cond_corr'])-np.average(hp_plot_df['cond_incorr']))
          /(sqrt((np.std(hp_plot_df['cond_corr'],ddof=1)) 
           **2+np.std(hp_plot_df['cond_incorr'],ddof = 1)**2)/2))
print 'Hippocampus Conditional Correct vs. Conditional Incorrect'
print stats.ttest_rel(hp_plot_df['cond_corr'], hp_plot_df['cond_incorr'])
print "Cohen's d = ", cohens_d

N = 2
conditions = ['Conditional Corr', 'Conditional Incorr']
hp_allsubjs = [hp_plot_df['cond_corr'], hp_plot_df['cond_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = hp_allsubjs, color = "#278fea", width = 0.3)
ax = sns.swarmplot(data = hp_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Hippocampal Activation")
plt.show()

# Medial PFC

In [ ]:
mpfc_plot_df

In [ ]:
for key in mpfc_plot_df:
    print "Mean {0}: ".format(key), np.mean(mpfc_plot_df['{0}'.format(key)])
    print "STD {0}: ".format(key), np.std(mpfc_plot_df['{0}'.format(key)])
    print ""
N = 4
conditions = ['Fixed Corr', 'Fixed Incorr', 'Conditional Corr', 'Conditional Incorr']
mpfc_allsubjs = [mpfc_plot_df['fixed_corr'], mpfc_plot_df['fixed_incorr'],
                 mpfc_plot_df['cond_corr'], mpfc_plot_df['cond_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (10,6))
ax = sns.boxplot(data = mpfc_allsubjs, color = "#f7a233", width = 0.3)
ax = sns.swarmplot(data = mpfc_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Medial PFC Activation")
plt.show()

### Medial PFC - Fixed vs. Conditional

In [ ]:
all_fixed = mpfc_plot_df[['fixed_corr', 'fixed_incorr']].mean(axis=1)
all_fixed = np.array(all_fixed)
all_cond = mpfc_plot_df[['cond_corr', 'cond_incorr']].mean(axis=1)
all_cond = np.array(all_cond)
print "Medial PFC FIXED", np.mean(all_fixed), np.std(all_fixed)
print "Medial PFC COND", np.mean(all_cond), np.std(all_cond)
print ""
print 'Medial PFC Fixed vs. Conditional'
print stats.ttest_rel(all_fixed, all_cond)
cohens_d = ((np.average(all_fixed) - np.average(all_cond)) / (sqrt((np.std(all_fixed, ddof = 1)) 
            ** 2 + np.std(all_cond, ddof = 1) ** 2) / 2))
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Fixed', 'Conditional']
mpfc_allsubjs = [all_fixed, all_cond]
ind = np.arange(N)
width = 0.5
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = mpfc_allsubjs, color = "#f7a233", width = 0.3)
ax = sns.swarmplot(data = mpfc_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Medial PFC Activation")
plt.show()

### Medial PFC Fixed - Correct vs. Incorrect

In [ ]:
cohens_d = ((np.average(mpfc_plot_df['fixed_corr']) - np.average(mpfc_plot_df['fixed_incorr']))
           / (sqrt((np.std(mpfc_plot_df['fixed_corr'], ddof=1)) 
           ** 2 + np.std(mpfc_plot_df['fixed_incorr'], ddof = 1) ** 2) / 2))
print 'Medial PFC Fixed Correct vs. Fixed Incorrect'
print stats.ttest_rel(mpfc_plot_df['fixed_corr'], mpfc_plot_df['fixed_incorr'])
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Fixed Corr', 'Fixed Incorr']
mpfc_allsubjs = [mpfc_plot_df['fixed_corr'], mpfc_plot_df['fixed_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = mpfc_allsubjs, color = "#f7a233", width = 0.3)
ax = sns.swarmplot(data = mpfc_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Medial PFC Activation")
plt.show()

### Medial PFC Conditional - Correct vs. Incorrect

In [ ]:
cohens_d = ((np.average(mpfc_plot_df['cond_corr']) - np.average(mpfc_plot_df['cond_incorr']))
            / (sqrt((np.std(mpfc_plot_df['cond_corr'], ddof=1)) 
            ** 2 + np.std(mpfc_plot_df['cond_incorr'], ddof = 1) ** 2) / 2))
print 'Medial PFC Conditional Correct vs. Conditional Incorrect'
print stats.ttest_rel(mpfc_plot_df['cond_corr'], mpfc_plot_df['cond_incorr'])
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Conditional Corr', 'Conditional Incorr']
mpfc_allsubjs = [mpfc_plot_df['cond_corr'], mpfc_plot_df['cond_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = mpfc_allsubjs, color = "#f7a233", width = 0.3)
ax = sns.swarmplot(data = mpfc_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Medial PFC Activation")
plt.show()

# Caudate

In [ ]:
caud_plot_df

In [ ]:
for key in caud_plot_df:
    print "Mean {0}: ".format(key), np.mean(caud_plot_df['{0}'.format(key)])
    print "STD {0}: ".format(key), np.std(caud_plot_df['{0}'.format(key)])
    print ""
N = 4
conditions = ['Fixed Corr', 'Fixed Incorr', 'Conditional Corr', 'Conditional Incorr']
caud_allsubjs = [caud_plot_df['fixed_corr'], caud_plot_df['fixed_incorr'],
                 caud_plot_df['cond_corr'], caud_plot_df['cond_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (10,6))
ax = sns.boxplot(data = caud_allsubjs, color = "#f9f96d", width = 0.3)
ax = sns.swarmplot(data = caud_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Caudate Activation")
plt.show()

### Caudate - Fixed vs. Conditional

In [ ]:
all_fixed = caud_plot_df[['fixed_corr', 'fixed_incorr']].mean(axis=1)
all_fixed = np.array(all_fixed)
all_cond = caud_plot_df[['cond_corr', 'cond_incorr']].mean(axis=1)
all_cond = np.array(all_cond)
print "Caudate FIXED", np.mean(all_fixed), np.std(all_fixed)
print "Caudate COND", np.mean(all_cond), np.std(all_cond)
print ""
print 'Caudate Fixed vs. Conditional'
print stats.ttest_rel(all_fixed, all_cond)
cohens_d = ((np.average(all_fixed) - np.average(all_cond)) / (sqrt((np.std(all_fixed, ddof = 1)) 
            ** 2 + np.std(all_cond, ddof = 1) ** 2) / 2))
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Fixed', 'Conditional']
caud_allsubjs = [all_fixed, all_cond]
ind = np.arange(N)
width = 0.5
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = caud_allsubjs, color = "#f9f96d", width = 0.3)
ax = sns.swarmplot(data = caud_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Caudate Activation")
plt.show()

### Caudate Fixed - Correct vs. Incorrect 

In [ ]:
cohens_d=((np.average(caud_plot_df['fixed_corr']) - np.average(caud_plot_df['fixed_incorr']))
          / (sqrt((np.std(caud_plot_df['fixed_corr'], ddof=1)) 
           ** 2 + np.std(caud_plot_df['fixed_incorr'], ddof = 1) ** 2) / 2))
print 'Caudate Fixed Correct vs. Fixed Incorrect'
print stats.ttest_rel(caud_plot_df['fixed_corr'], caud_plot_df['fixed_incorr'])
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Fixed Corr', 'Fixed Incorr']
caud_allsubjs = [caud_plot_df['fixed_corr'], caud_plot_df['fixed_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = caud_allsubjs, color = "#f9f96d", width = 0.3)
ax = sns.swarmplot(data = caud_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Caudate Activation")
plt.show()

### Caudate Conditional - Correct vs. Incorrect

In [ ]:
cohens_d=((np.average(caud_plot_df['cond_corr']) - np.average(caud_plot_df['cond_incorr']))
          / (sqrt((np.std(caud_plot_df['cond_corr'], ddof=1)) 
           ** 2 + np.std(caud_plot_df['cond_incorr'], ddof = 1) ** 2) / 2))
print 'Caudate Conditional Correct vs. Conditional Incorrect'
print stats.ttest_rel(caud_plot_df['cond_corr'], caud_plot_df['cond_incorr'])
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Conditional Corr', 'Conditional Incorr']
caud_allsubjs = [caud_plot_df['cond_corr'], caud_plot_df['cond_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = caud_allsubjs, color = "#f9f96d", width = 0.3)
ax = sns.swarmplot(data = caud_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Caudate Activation")
plt.show()

# Putamen

In [ ]:
put_plot_df

In [ ]:
for key in put_plot_df:
    print "Mean {0}: ".format(key), np.mean(put_plot_df['{0}'.format(key)])
    print "STD {0}: ".format(key), np.std(put_plot_df['{0}'.format(key)])
    print ""
N = 4
conditions = ['Fixed Corr', 'Fixed Incorr', 'Conditional Corr', 'Conditional Incorr']
put_allsubjs = [put_plot_df['fixed_corr'], put_plot_df['fixed_incorr'],
                put_plot_df['cond_corr'], put_plot_df['cond_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (10,6))
ax = sns.boxplot(data = put_allsubjs, color = "#c34aef", width = 0.3)
ax = sns.swarmplot(data = put_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Putamen Activation")
plt.show()

### Putamen - Fixed vs. Conditional

In [ ]:
all_fixed = put_plot_df[['fixed_corr', 'fixed_incorr']].mean(axis=1)
all_fixed = np.array(all_fixed)
all_cond = put_plot_df[['cond_corr', 'cond_incorr']].mean(axis=1)
all_cond = np.array(all_cond)
print "Putamen FIXED", np.mean(all_fixed), np.std(all_fixed)
print "Putamen COND", np.mean(all_cond), np.std(all_cond)
print ""
print 'Putamen Fixed vs. Conditional'
print stats.ttest_rel(all_fixed, all_cond)
cohens_d = ((np.average(all_fixed) - np.average(all_cond)) / (sqrt((np.std(all_fixed, ddof = 1)) 
            ** 2 + np.std(all_cond, ddof = 1) ** 2) / 2))
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Fixed', 'Conditional']
put_allsubjs = [all_fixed, all_cond]
ind = np.arange(N)
width = 0.5
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = put_allsubjs, color = "#c34aef", width = 0.3)
ax = sns.swarmplot(data = put_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Putamen Activation")
plt.show()

### Putamen Fixed - Correct vs. Incorrect

In [ ]:
cohens_d = ((np.average(put_plot_df['fixed_corr']) - np.average(put_plot_df['fixed_incorr']))
           / (sqrt((np.std(put_plot_df['fixed_corr'], ddof=1)) 
           ** 2 + np.std(put_plot_df['fixed_incorr'], ddof = 1) ** 2) / 2))
print 'Putamen Fixed Correct vs. Fixed Incorrect'
print stats.ttest_rel(put_plot_df['fixed_corr'], put_plot_df['fixed_incorr'])
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Fixed Corr', 'Fixed Incorr']
put_allsubjs = [put_plot_df['fixed_corr'], put_plot_df['fixed_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = put_allsubjs, color = "#c34aef", width = 0.3)
ax = sns.swarmplot(data = put_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Putamen Activation")
plt.show()

### Putamen Conditional - Correct vs. Incorrect

In [ ]:
cohens_d = ((np.average(put_plot_df['cond_corr']) - np.average(put_plot_df['cond_incorr']))
            / (sqrt((np.std(put_plot_df['cond_corr'], ddof=1)) 
            ** 2 + np.std(put_plot_df['cond_incorr'], ddof = 1) ** 2) / 2))
print 'Putamen Conditional Correct vs. Conditional Incorrect'
print stats.ttest_rel(put_plot_df['cond_corr'], put_plot_df['cond_incorr'])
print "Cohen's d = ", cohens_d
N = 2
conditions = ['Conditional Corr', 'Conditional Incorr']
put_allsubjs = [put_plot_df['cond_corr'], put_plot_df['cond_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (5,5))
ax = sns.boxplot(data = put_allsubjs, color = "#c34aef", width = 0.3)
ax = sns.swarmplot(data = put_allsubjs, color='.25')
ax.set_xticklabels(conditions)
ax.set_xticks(ind)
ax.set_ylabel("Arbitrary units")
ax.set_title("Putamen Activation")
plt.show()

# Correlation Scatterplots

In [ ]:
all_prop_fixed = []
all_prop_cond = []

for sub in subs:
    sub_dir = '/home/data/madlab/data/mri/wmaze/scanner_behav/{0}/'.format(sub)
    dir_file = glob(join(sub_dir, '{0}_wmazebl_2015*.txt'.format(sub)))   
    dir_file.sort()    
    
    data_run1 = pd.read_table(dir_file[0])
    data_run1 = data_run1[:-3]
    data_run2 = pd.read_table(dir_file[1])
    data_run2 = data_run2[:-3]
    data_run3 = pd.read_table(dir_file[2])
    data_run3 = data_run3[:-3]
    data_run4 = pd.read_table(dir_file[3])
    data_run4 = data_run4[:-3]
    data_run5 = pd.read_table(dir_file[4])
    data_run5 = data_run5[:-3]
    data_run6 = pd.read_table(dir_file[5])
    data_run6 = data_run6[:-3]
    
    all_runs = [data_run1, data_run2, data_run3, data_run4, data_run5, data_run6]    
    data_set = pd.concat(all_runs)    
    trialtype = data_set['TrialType'].values
    correct = data_set['Correct'].values 
    
    fixed_corr = np.where((correct == 1) & ((trialtype == 'A') | (trialtype == 'C')))[0]
    fixed_incorr = np.where((correct == 0) & ((trialtype == 'A') | (trialtype == 'C')))[0]
    cond_corr = np.where((trialtype == 'B') & (correct == 1)) [0] 
    cond_incorr = np.where((trialtype == 'B') & (correct == 0)) [0] 
     
    all_prop_fixed.append(len(fixed_corr)/float(len(fixed_corr) + len(fixed_incorr)))
    all_prop_cond.append(len(cond_corr)/float(len(cond_corr) + len(cond_incorr)))
    
all_data_df['prop_fixed'] = all_prop_fixed
all_data_df['prop_cond'] = all_prop_cond

In [ ]:
region = ['hp', 'mpfc', 'caud', 'put']
for r in region:
    all_data_df['{0}_all_fixed_4'.format(r,l)] = (all_data_df['lh{0}_fixed_corr'.format(r)] 
                                                  + all_data_df['rh{0}_fixed_corr'.format(r)]
                                                  + all_data_df['lh{0}_fixed_incorr'.format(r)] 
                                                  + all_data_df['rh{0}_fixed_incorr'.format(r)])/4.
    all_data_df['{0}_all_cond_4'.format(r,l)] = (all_data_df['lh{0}_fixed_corr'.format(r)] 
                                                 + all_data_df['rh{0}_cond_corr'.format(r)]
                                                 + all_data_df['lh{0}_cond_incorr'.format(r)] 
                                                 + all_data_df['rh{0}_cond_incorr'.format(r)])/4.

## Hippocampus

In [ ]:
x = all_data_df['prop_fixed']
y = all_data_df['hp_all_fixed_4']

print 'FIXED', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_fixed', y = 'hp_all_fixed_4', data = all_data_df)
ax.set_ylabel('HPC Activation')
ax.set_xlabel('Avg Performance')
ax.set_title('HPC FIXED')
plt.show()

In [ ]:
x = all_data_df['prop_cond']
y = all_data_df['hp_all_cond_4']

print 'CONDITIONAL', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_cond', y = 'hp_all_cond_4', data = all_data_df)
ax.set_ylabel('HPC Activation')
ax.set_xlabel('Avg Performance')
ax.set_title('HPC CONDITIONAL')
plt.show()

## Medial PFC

In [ ]:
x = all_data_df['prop_fixed']
y = all_data_df['mpfc_all_fixed_4']

print 'FIXED', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_fixed', y = 'mpfc_all_fixed_4', data = all_data_df, color = "#fc8b02")
ax.set_ylabel('mPFC Activation')
ax.set_xlabel('Avg Performance')
ax.set_title('Medial PFC FIXED')
plt.show()

In [ ]:
x = all_data_df['prop_fixed']
y = all_data_df['mpfc_all_cond_4']

print 'CONDITIONAL', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_cond', y = 'mpfc_all_cond_4', data = all_data_df, color = "#fc8b02")
ax.set_ylabel('mPFC Activation')
ax.set_xlabel('Avg Performance')
ax.set_title('Medial PFC CONDITIONAL')
plt.show()

## Caudate

In [ ]:
x = all_data_df['prop_fixed']
y = all_data_df['caud_all_fixed_4']

print 'FIXED', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_fixed', y = 'caud_all_fixed_4', data = all_data_df, color = "#f9c402")
ax.set_ylabel('Caudate Activation')
ax.set_xlabel('Avg Performance')
ax.set_title('Caudate FIXED')
plt.show()

In [ ]:
x = all_data_df['prop_fixed']
y = all_data_df['caud_all_cond_4']

print 'CONDITIONAL', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_cond', y = 'caud_all_cond_4', data = all_data_df, color = "#f9c402")
ax.set_ylabel('Caudate Activation')
ax.set_xlabel('Avg Performance')
ax.set_title('Caudate CONDITIONAL')
plt.show()

## Putamen

In [ ]:
x = all_data_df['prop_fixed']
y = all_data_df['put_all_fixed_4']

print 'FIXED', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_fixed', y = 'put_all_fixed_4', data = all_data_df, color = "#c34aef")
ax.set_ylabel('Putamen Activation')
ax.set_xlabel('Avg Performance')
ax.set_title('Putamen FIXED')
plt.show()

In [ ]:
x = all_data_df['prop_fixed']
y = all_data_df['put_all_cond_4']

print 'CONDITIONAL', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_cond', y = 'put_all_cond_4', data = all_data_df, color = "#c34aef")
ax.set_ylabel('Putamen Activation')
ax.set_xlabel('Avg Performance')
ax.set_title('Putamen CONDITIONAL')
plt.show()